<a href="https://colab.research.google.com/github/dnaligase/PsychoPy_OdorBCI_shared/blob/main/KNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#install mne, autoreject
!pip install mne
!pip install autoreject

     |████████████████████████████████| 7.0MB 5.2MB/s 


In [ ]:
#install libraries for our code
import os, mne, glob, time
import pandas as pd
import numpy  as np
import matplotlib.pyplot as plt
import math
import warnings

from tqdm import tqdm
from scipy import signal, special
from autoreject import AutoReject
from itertools import combinations

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RepeatedKFold

from google.colab import drive
print('Libraries have been imported.')

Libraries have been imported


In [ ]:
# open Google Drive via the link. Sign in if neccessary. Copy and paste the key to cell below to grat access Google Drive folders
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
# First add a shortcut to your drive. Open shared folder "EEG_sliced". http://bit.ly/EEG_sliced
# Click an arrow right to the foldername. From the dropdown menu choose "Add a shortcut to Drive".
# Shared folder "EEG_sliced" with the EEG data will appear in your Google Drive folder
folder_path="/content/gdrive/MyDrive/Aspir/Olfactory-Dataset/New_data"
output_folder="/content/gdrive/MyDrive/EEG_sliced"
print('Folders has been set.')

Folders has been set


In [ ]:
#set the code of the participant. Leave blank("") in order to use all files which name contain "filt"
code=""

#let the code set the filename
if code!='':
  filename=code+'*filt*.fif'
else:
  filename='*filt*.fif'
print(f'Code will look for {filename} file(s).')


Code will look for *filt*.fif file(s).


# Function Space
You will find the functions to work through the pipeline under the hood.

In [ ]:
def prepare_epochs(folder_path=str, tmin=0., tmax=2., l_freq=None, h_freq=20.,
                   onset = 'inhale', exclude=[], save=False, filename="*ilt*.fif", output_folder="/content/gdrive/MyDrive/EEG_sliced"):
  # CAN BE USED: exclude=["Fz", "Fp1", "Fp2", "P4", "T4", "T5", "T3", "T6"]
  
  """
  Takes path to folder with .fif files as input. Outputs the list of tuples,
  where each tuple contains the (1) EEG epochs per odor, (2) odor name,
  (3) corresponding label, (4) subject id, (5) total number of epochs per odor.

  Parameters
  ----------
    folder_path : str
        Folder with EEG files, by default str
    tmin : float, optional
         Starting point for the epoch, by default 0.0
    tmax : float, optional
        Endpoint for the epoch, by default 2.0
    l_freq : list, optional
        Low frequency filter, by default None
    h_freq : float, optional
        High frequency filter, by default 20.0
    onset : str, optional
        Epoch onset settings button/inhale, by default "inhale"
    exclude : list, optional
        Channels to be excluded, by default []
    save : bool
        Save epochs into new file, by default False
    filename : str
        Name of the file(s) to be sliced, by default "*Filtered*.fif"
    output_folder: str
        Path to the output folder. By deafult "/content/gdrive/MyDrive/EEG_Sliced"

  Returns
  -------
  epochs_sliced : list
    List of tuples. See the docstring.
  ch_names : list
    List of channel names.
  """

  # create an AutoReject object
  ar = AutoReject(verbose=False)

  # create epochs data
  epochs_sliced = []

  rel_path = os.path.abspath("")

  # Load list of raw EEG filenames
  fnames = glob.glob(os.path.join(rel_path, folder_path, filename))

  print(f"Searching for {filename}")
  print(f"\nWORKING: Preparing {len(fnames)} filtered EEG raw file(s). This may take a while.")

  for fname in fnames:
      print(f"WORKING: Slicing {os.path.basename(fname)}\n")
      raw = mne.io.read_raw_fif(fname, preload=True, verbose=False).pick(picks=None, exclude=exclude)
      raw.filter(l_freq=l_freq, h_freq=h_freq, verbose=False)

      # re-reference to 'common average'
      mne.set_eeg_reference(raw, ref_channels='average', verbose=False)

      # set onset inhale or button
      if onset == 'inhale':
        onset_regexp='^(?![Ii]|[Nn][Aa]|[Cc][Oo][Ff][Ff]|[Vv][Aa][Nn][Ii]|[Cc][Ii][Tt][Rr]|[Ww][Aa][Tt][Ee]|[Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$'
        output_name=os.path.basename(fname)[:9]+"_inhale-epo.fif"
      elif onset == 'button':
        onset_regexp='^(?![Ii]|[Nn][Aa]|[Cc][Oo][Ff][_]|[Vv][Aa][Nn][_]|[Cc][Ii][Tt][_]|[Ww][Aa][Tt][_]|[Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$'
        output_name=os.path.basename(fname)[:9]+"_button-epo.fif"
      else:
        # set button as default, print warning
        onset_regexp='^(?![Ii]|[Nn][Aa]|[Cc][Oo][Ff][Ff]|[Vv][Aa][Nn][Ii]|[Cc][Ii][Tt][Rr]|[Ww][Aa][Tt][Ee]|[Bb][Aa][Dd]|[Ee][Dd][Gg][Ee]).*$'
        output_name=os.path.basename(fname)[:9]+"_button-epo.fif"
        warnings.warn(f'\n Could not recognize {onset}. Consider "inhale" or "button" next time.\nSetting to "inhale".', RuntimeWarning)

      # extract inhalation events
      events, event_id = mne.events_from_annotations(raw, regexp=onset_regexp)
      print('\n\n {} events found.'.format(len(events)))

      # slicing epochs
      print("\nWORKING: Slicing epochs with {} as an onset.  Time interval: {} - {}. Frequencies: {} - {}".format(onset, tmin, tmax, l_freq, h_freq))
      epochs_mne = mne.Epochs(raw, events, event_id, tmin=tmin, tmax=tmax, baseline=None, verbose=False)

      # clean the epochs using AAR algorithm
      epochs_clean = ar.fit_transform(epochs_mne.load_data())

      for odor, idx in event_id.items():
          epochs, labels = epochs_clean[odor].get_data(), epochs_clean[odor].events[:, -1]
          
          # create a list of tuples; normalize each trial by its std
          trial = epochs / np.expand_dims(np.std(epochs, axis=-1), axis=-1), \
              odor, labels - 1, os.path.basename(fname)[:9], (events[:, 2] == idx).sum()

          epochs_sliced.append(trial)

      if save:
        epochs_clean.save(os.path.join(output_folder,output_name))
      else:
        pass

  print('\n ------DONE-----')

  return epochs_sliced, epochs_clean.ch_names


# define auxillary functions
def average(a: list):
    return sum(a) / len(a)

def feats_reorder(ch_no=int):
  """
  Reorder consecutive elements, e. g. [1, 2, 3, 4] -> [1, 3, 2, 4], if there
  are n channels and 2 features per channel.
  """
  idxs = np.arange(ch_no * 2)
  idxs_reordered = np.insert(idxs[ch_no:], np.arange(len(idxs[:ch_no])), idxs[:ch_no])

  return idxs_reordered 

print('All functions are successfully defined.')  

All functions are successfully defined.


# KNN
KNN implementation based on two features (StDev and Average of Wavelet coefficients) per channel.

In [ ]:
# crate epochs and channel names. get data, suitable for our problem
epochs_sliced, ch_names = prepare_epochs(folder_path=folder_path, onset='button',tmin=-5, tmax=0., save=False, filename=filename, h_freq=None)

NameError: ignored

In [ ]:
# how many classes we have
odors_no = 4

# total number of subjects used
subjects_no = len(epochs_sliced) // odors_no
# create list of odor pair tuples
combs = list(combinations([i for i in range(odors_no)], r=2))

# k-NN classifier
accuracies = np.zeros((subjects_no, special.comb(4, 2, exact=True)))
neigh = KNeighborsClassifier(n_neighbors=13)
d = {"subj_id": [], "age": [], "sex": [], "odor_1_categ": [], "odor_2_categ": [], "odor_1": [], "odor_2": [], "best_acc": [], "best_chan": []}

# first, select a subject
for subject_id in tqdm(range(subjects_no)):
    pairwise_scores = []
    # second, select an odor pair
    for j, k in tqdm(combs):

        epochs_paired = np.concatenate([epochs_sliced[subject_id * odors_no + j][0], \
            epochs_sliced[subject_id * odors_no + k][0]])

        labels_paired = np.concatenate([epochs_sliced[subject_id * odors_no + j][2], \
            epochs_sliced[subject_id * odors_no + k][2]])

        # here we calculate wavelet-transform coefficients (WTC) in the alpha-band 8-13 Hz
        coefs = signal.cwt(epochs_paired[:, :, :].flatten(), signal.morlet2, np.arange(8, 13))

        # as the output is imaginary, let's find an absolute values
        # also averaging CWT in a band
        coefs_band = abs(coefs.mean(axis=0))

        # returning initial shape of an array
        coefs_band = coefs_band.reshape(epochs_paired.shape[0], epochs_paired.shape[1], -1)
        # finding average and st. dev. of WTC
        A_wtc, S_wtc = coefs_band.mean(axis=-1), np.std(coefs_band, axis=-1)
        
        # stack features so that for single trial they are:
        # [A_wtc_ch1, A_wtc_ch2, A_wtc_ch3, S_wtc_ch1, S_wtc_ch2, S_wtc_ch3]
        wtc_final = np.hstack((A_wtc, S_wtc))

        # reoder features, e. g. [A_wtc_ch1, S_wtc_ch1, ...]
        idxs = feats_reorder(len(ch_names))
        wtc_final_reorder = wtc_final[:, idxs]

        ch_scores = []
        per_chan_std = []
        # third, for each individual and each odor pair calculate prediction accuracy for each channel
        for ch in range(len(ch_names)):
        
            testscore = []
            trainscore = []
            
            X = wtc_final_reorder[:, ch*2 : (ch + 1) * 2]
            y = labels_paired
            kf = RepeatedKFold(n_splits=5, n_repeats=20)
            

            for train_index, test_index in kf.split(X):
                X_train, X_test = X[train_index], X[test_index]
                y_train, y_test = y[train_index], y[test_index]

                neigh.fit(X_train, y_train)
                testscore.append(neigh.score(X_test, y_test))
                trainscore.append(neigh.score(X_train, y_train))

            # per_chan_std.append(np.std(ch_scores))
            ch_scores.append(np.round(average(testscore), decimals=3))
        
        # create dictionary of subject id, odor1, odor2, max accuracy across the chs, best ch name
        d['subj_id'].append(epochs_sliced[subject_id * odors_no][3])
        d['age'].append(epochs_sliced[subject_id * odors_no][3][-3:-1])
        d['sex'].append(epochs_sliced[subject_id * odors_no][3][-1])
        d['odor_1_categ'].append(epochs_sliced[j][1])
        d['odor_2_categ'].append(epochs_sliced[k][1])
        d['odor_1'].append(j)
        d['odor_2'].append(k)
        d['best_acc'].append(np.max(ch_scores))
        # d['std'].append(per_chan_std[np.argmax(ch_scores)])
        d['best_chan'].append(ch_names[np.argmax(ch_scores)])

        pairwise_scores.append(np.max(ch_scores))

    accuracies[subject_id, :] = pairwise_scores
print("\n---DONE---")
print(accuracies)


  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:05<00:29,  5.86s/it]

 33%|███▎      | 2/6 [00:14<00:26,  6.69s/it]

 50%|█████     | 3/6 [00:23<00:21,  7.33s/it]

 67%|██████▋   | 4/6 [00:28<00:13,  6.81s/it]

 83%|████████▎ | 5/6 [00:34<00:06,  6.49s/it]

100%|██████████| 6/6 [00:40<00:00,  6.73s/it]

  6%|▋         | 1/16 [00:40<10:06, 40.41s/it]

  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:08<00:40,  8.07s/it]

 33%|███▎      | 2/6 [00:16<00:32,  8.16s/it]

 50%|█████     | 3/6 [00:24<00:24,  8.19s/it]

 67%|██████▋   | 4/6 [00:33<00:16,  8.27s/it]

 83%|████████▎ | 5/6 [00:41<00:08,  8.33s/it]

100%|██████████| 6/6 [00:50<00:00,  8.38s/it]

 12%|█▎        | 2/16 [01:30<10:07, 43.37s/it]

  0%|          | 0/6 [00:00<?, ?it/s]

 17%|█▋        | 1/6 [00:10<00:50, 10.19s/it]

 33%|███▎      | 2/6 [00:20<00:40, 10.19s/it]

 50%|█████     | 3/6 [00:30<00:30, 10.17s/it]

 67%|██████▋   | 4/6 [00:40<00:20, 10.19s


---DONE---
[[0.6   0.518 0.516 0.387 0.547 0.495]
 [0.596 0.657 0.548 0.632 0.61  0.561]
 [0.6   0.669 0.735 0.705 0.677 0.688]
 [0.606 0.614 0.627 0.507 0.621 0.598]
 [0.577 0.574 0.618 0.612 0.64  0.608]
 [0.627 0.659 0.64  0.598 0.55  0.622]
 [0.603 0.58  0.679 0.605 0.582 0.608]
 [0.608 0.657 0.558 0.588 0.665 0.589]
 [0.576 0.565 0.556 0.645 0.625 0.605]
 [0.586 0.611 0.636 0.627 0.561 0.634]
 [0.578 0.563 0.561 0.536 0.643 0.654]
 [0.509 0.554 0.649 0.52  0.605 0.694]
 [0.627 0.692 0.609 0.668 0.617 0.68 ]
 [0.634 0.554 0.598 0.563 0.6   0.646]
 [0.593 0.617 0.773 0.599 0.62  0.726]
 [0.681 0.656 0.676 0.582 0.584 0.576]]


In [ ]:
table = pd.DataFrame(d)
table

,subj_id,age,sex,odor_1_categ,odor_2_categ,odor_1,odor_2,best_acc,best_chan
0,1R912_39F,39,F,cit_inhale,cof_inhale,0,1,0.600,C4
1,1R912_39F,39,F,cit_inhale,van_inhale,0,2,0.518,P3
2,1R912_39F,39,F,cit_inhale,wat_inhale,0,3,0.516,T3
3,1R912_39F,39,F,cof_inhale,van_inhale,1,2,0.387,Cz
4,1R912_39F,39,F,cof_inhale,wat_inhale,1,3,0.547,O2
...,...,...,...,...,...,...,...,...,...
91,ZU9B3_34M,34,M,cit_inhale,van_inhale,0,2,0.656,T4
92,ZU9B3_34M,34,M,cit_inhale,wat_inhale,0,3,0.676,Fp2
93,ZU9B3_34M,34,M,cof_inhale,van_inhale,1,2,0.582,T6
94,ZU9B3_34M,34,M,cof_inhale,wat_inhale,1,3,0.584,T3
